In [151]:
import pandas as pd

In [156]:
feeding_data = pd.read_csv(r"C:\Users\sarah\OneDrive - KU Leuven\Desktop\Data Visualization\Project\data\Exp1 - Feeding data.csv")
feeding_data.head()

In [157]:
pigs_data = pd.read_csv(r"C:\Users\sarah\OneDrive - KU Leuven\Desktop\Data Visualization\Project\data\Exp1 - Pig registration all info combined.csv")
pigs_data.head()

In [158]:
## DATA CLEANING

# Delete the rows where 'pig' = 0 or where 'pig' from feeding_data is not in 'pig' from pigs
feeding_data = feeding_data[feeding_data['pig'] != 0]
feeding_data = feeding_data[feeding_data['pig'].isin(pigs_data['pig'])]

# Delete rows where at least one of 'intake', 'duration' or 'rate' contain negative values
feeding_data = feeding_data[(feeding_data['intake'] >= 0) & (feeding_data['duration'] >= 0) & (feeding_data['rate'] >= 0)]

# Convert 'date', 'start' and 'end' columns to datetime
feeding_data['date'] = pd.to_datetime(feeding_data['date'])
feeding_data['start'] = pd.to_datetime(feeding_data['start'])
feeding_data['end'] = pd.to_datetime(feeding_data['end'])

# Add an index column to preserve the original order
feeding_data['original_index'] = feeding_data.index

## VARIABLES PER PIG

# Calculate the total intake per pig 
feeding_data['tot_intake_pig'] = feeding_data.groupby(['pig'])['intake'].transform('sum')

# Calculate the total intake per pig per day
feeding_data['tot_intake_day_pig'] = feeding_data.groupby(['pig', 'date'])['intake'].transform('sum')

# Calculate the average rate per pig per day
feeding_data['avg_rate_day_pig'] = feeding_data.groupby(['pig', 'date'])['rate'].transform('mean')

# Calculate the average duration per pig per day
feeding_data['avg_duration_day_pig'] = feeding_data.groupby(['pig', 'date'])['duration'].transform('mean')

# Calculate the total intake per pig per hour
feeding_data['tot_intake_hour_pig'] = feeding_data.groupby(['pig', 'date', 'hour'])['intake'].transform('sum')

# Calculate the average rate per pig per hour
feeding_data['avg_rate_hour_pig'] = feeding_data.groupby(['pig', 'date', 'hour'])['rate'].transform('mean')

# Calculate the average duration per pig per hour
feeding_data['avg_duration_hour_pig'] = feeding_data.groupby(['pig', 'date', 'hour'])['duration'].transform('mean')

## VARIABLES PER STATION

# Calculate the total intake per station 
feeding_data['tot_intake_station'] = feeding_data.groupby(['station'])['intake'].transform('sum')

# Calculate the total intake per station per day
feeding_data['tot_intake_day_station'] = feeding_data.groupby(['station', 'date'])['intake'].transform('sum')

# Calculate the average rate per station per day
feeding_data['avg_rate_day_station'] = feeding_data.groupby(['station', 'date'])['rate'].transform('mean')

# Calculate the average duration per station per day
feeding_data['avg_duration_day_station'] = feeding_data.groupby(['station', 'date'])['duration'].transform('mean')

# Calculate the total intake per station per hour
feeding_data['tot_intake_hour_station'] = feeding_data.groupby(['station', 'date', 'hour'])['intake'].transform('sum')

# Calculate the average rate per station per hour
feeding_data['avg_rate_hour_station'] = feeding_data.groupby(['station', 'date', 'hour'])['rate'].transform('mean')

# Calculate the average duration per station per hour
feeding_data['avg_duration_hour_station'] = feeding_data.groupby(['station', 'date', 'hour'])['duration'].transform('mean')

# Display the first few rows of the updated DataFrame
feeding_data.head()

In [159]:
# BEFORE/AFTER VARIABLES 

# Calculate the time before and after in seconds
feeding_data['time_before'] = (feeding_data['start'] - feeding_data['end'].shift(1)).dt.total_seconds()
feeding_data['time_after'] = (feeding_data['start'].shift(-1) - feeding_data['end']).dt.total_seconds()

## BEFORE/AFTER VARIABLES PER PIG

# Sorting per pig and start time
feeding_data = feeding_data.sort_values(by=['pig', 'start'], ascending=True)

# Shift the 'pig' column to compare with the current row
feeding_data['h_pig_before'] = feeding_data['pig'].shift(1)
feeding_data['h_pig_after'] = feeding_data['pig'].shift(-1)

# Calculate the time before and after in seconds per pig
feeding_data['time_before_pig'] = (feeding_data['start'] - feeding_data['end'].shift(1)).dt.total_seconds()
feeding_data['time_after_pig'] = (feeding_data['start'].shift(-1) - feeding_data['end']).dt.total_seconds()
feeding_data['time_before_pig'] = feeding_data.apply(lambda row: int(row['time_before_pig']) if row['pig'] == row['h_pig_before'] else '', axis=1)
feeding_data['time_after_pig'] = feeding_data.apply(lambda row: int(row['time_after_pig']) if row['pig'] == row['h_pig_after'] else '', axis=1)

# Drop the helper columns
feeding_data = feeding_data.drop(columns=['h_pig_before', 'h_pig_after'])

## BEFORE/AFTER VARIABLES PER STATION

# Sorting per station and start time
feeding_data = feeding_data.sort_values(by=['station', 'start'], ascending=True)

# Shift the 'station' column to compare with the current row
feeding_data['h_station_before'] = feeding_data['station'].shift(1)
feeding_data['h_station_after'] = feeding_data['station'].shift(-1)

# Calculate the time before and after in seconds per station
feeding_data['time_before_station'] = (feeding_data['start'] - feeding_data['end'].shift(1)).dt.total_seconds()
feeding_data['time_after_station'] = (feeding_data['start'].shift(-1) - feeding_data['end']).dt.total_seconds()
feeding_data['time_before_station'] = feeding_data.apply(lambda row: int(row['time_before_station']) if row['station'] == row['h_station_before'] else '', axis=1)
feeding_data['time_after_station'] = feeding_data.apply(lambda row: int(row['time_after_station']) if row['station'] == row['h_station_after'] else '', axis=1)

# Get the pig before and after per station
feeding_data['pig_before_station'] = feeding_data['pig'].shift(1)
feeding_data['pig_after_station'] = feeding_data['pig'].shift(-1)
feeding_data['pig_before_station'] = feeding_data.apply(lambda row: int(row['pig_before_station']) if row['station'] == row['h_station_before'] else '', axis=1)
feeding_data['pig_after_station'] = feeding_data.apply(lambda row: int(row['pig_after_station']) if row['station'] == row['h_station_after'] else '', axis=1)

# Create pig_index column 
feeding_data = feeding_data.merge(pigs_data[['pig', 'pig_index']], on='pig', how='left')

# Get the pig before and after per station
feeding_data['pig_before_index'] = feeding_data['pig_index'].shift(1)
feeding_data['pig_after_index'] = feeding_data['pig_index'].shift(-1)
feeding_data['pig_before_index'] = feeding_data.apply(lambda row: int(row['pig_before_index']) if row['station'] == row['h_station_before'] else '', axis=1)
feeding_data['pig_after_index'] = feeding_data.apply(lambda row: int(row['pig_after_index']) if row['station'] == row['h_station_after'] else '', axis=1)

# Drop the helper columns
feeding_data = feeding_data.drop(columns=['h_station_before', 'h_station_after'])

# Convert columns to numeric and round
feeding_data['time_before'] = pd.to_numeric(feeding_data['time_before'], errors='coerce')
feeding_data['time_after'] = pd.to_numeric(feeding_data['time_after'], errors='coerce')
feeding_data['time_before'] = feeding_data['time_before'].round()
feeding_data['time_after'] = feeding_data['time_after'].round()
feeding_data['time_before_pig'] = pd.to_numeric(feeding_data['time_before_pig'], errors='coerce')
feeding_data['time_after_pig'] = pd.to_numeric(feeding_data['time_after_pig'], errors='coerce')
feeding_data['time_before_pig'] = feeding_data['time_before_pig'].round()
feeding_data['time_after_pig'] = feeding_data['time_after_pig'].round()
feeding_data['time_before_station'] = pd.to_numeric(feeding_data['time_before_station'], errors='coerce')
feeding_data['time_after_station'] = pd.to_numeric(feeding_data['time_after_station'], errors='coerce')
feeding_data['time_before_station'] = feeding_data['time_before_station'].round()
feeding_data['time_after_station'] = feeding_data['time_after_station'].round()
feeding_data['pig_before_station'] = feeding_data['pig_before_station'].astype('int64', errors='ignore')
feeding_data['pig_after_station'] = feeding_data['pig_after_station'].astype('int64', errors='ignore')
feeding_data['pig_before_station'] = feeding_data['pig_before_station'].round()
feeding_data['pig_after_station'] = feeding_data['pig_after_station'].round()

# Remove negative values by setting them to NaN
feeding_data['time_before'] = feeding_data['time_before'].where(feeding_data['time_before'] >= 0)
feeding_data['time_after'] = feeding_data['time_after'].where(feeding_data['time_after'] >= 0)
feeding_data['time_before_pig'] = feeding_data['time_before_pig'].where(feeding_data['time_before_pig'] >= 0)
feeding_data['time_after_pig'] = feeding_data['time_after_pig'].where(feeding_data['time_after_pig'] >= 0)
feeding_data['time_before_station'] = feeding_data['time_before_station'].where(feeding_data['time_before_station'] >= 0)
feeding_data['time_after_station'] = feeding_data['time_after_station'].where(feeding_data['time_after_station'] >= 0)

# Restore the original order
feeding_data = feeding_data.sort_values(by=['original_index'])

# Drop the index column
feeding_data = feeding_data.drop(columns=['original_index'])

# Display the first few rows of the updated DataFrame
feeding_data.head()

In [160]:
# Write the DataFrame to a CSV file
feeding_data.to_csv(r"C:\Users\sarah\OneDrive - KU Leuven\Desktop\Data Visualization\Project\data\feeding_data.csv", index=False)